In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [3]:
path = r"C:\Users\Maryla\Desktop\MY_PROJECTS\excel files\CycleSalesWorksheets.xlsx"
data=pd.read_excel(path, sheet_name="Transactions")


In [4]:
data.head(5)


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [7]:
data.describe()


,transaction_id,product_id,customer_id,online_order,list_price,standard_cost,product_first_sold_date
count,20000.000000,20000.00000,20000.000000,19640.000000,20000.000000,19803.000000,19803.000000
mean,10000.500000,45.36465,1738.246050,0.500458,1107.829449,556.046951,38199.776549
std,5773.647028,30.75359,1011.951046,0.500013,582.825242,405.955660,2875.201110
min,1.000000,0.00000,1.000000,0.000000,12.010000,7.210000,33259.000000
25%,5000.750000,18.00000,857.750000,0.000000,575.270000,215.140000,35667.000000
50%,10000.500000,44.00000,1736.000000,1.000000,1163.890000,507.580000,38216.000000
75%,15000.250000,72.00000,2613.000000,1.000000,1635.300000,795.100000,40672.000000
max,20000.000000,100.00000,5034.000000,1.000000,2091.470000,1759.850000,42710.000000


In [8]:
data.dtypes


transaction_id                      int64
product_id                          int64
customer_id                         int64
transaction_date           datetime64[ns]
online_order                      float64
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date           float64
dtype: object

In [12]:
list1 = ["brand", "product_line", "product_class","product_size"]
for x in list1:
    print (data[x].value_counts())
    print ()

Solex             4253
Giant Bicycles    3312
WeareA2B          3295
OHM Cycles        3043
Trek Bicycles     2990
Norco Bicycles    2910
Name: brand, dtype: int64

Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: product_line, dtype: int64

medium    13826
high       3013
low        2964
Name: product_class, dtype: int64

medium    12990
large      3976
small      2837
Name: product_size, dtype: int64



#### When using group by vs pivot table on one category (index) both methods are equal!

In [15]:
data.pivot_table(index ="brand", values = "list_price")

# OR using groupby
data.groupby("brand")["list_price"].mean()

,list_price
brand,
Giant Bicycles,1235.407204
Norco Bicycles,913.202450
OHM Cycles,983.706983
Solex,1057.271956
Trek Bicycles,1183.783398
WeareA2B,1263.454813


In [20]:
data.pivot_table(index ="brand", values = "list_price", aggfunc = max) 

# OR using groupby
data.groupby("brand")["list_price"].max()

,list_price
brand,
Giant Bicycles,1977.36
Norco Bicycles,1661.92
OHM Cycles,2005.66
Solex,2083.94
Trek Bicycles,2091.47
WeareA2B,1992.93


### But when we start using two categories to group - this is where we start to see differences in both methods.
#### pivot_table is better organized and we can clearly see where there was no sales such as in Giant brand mountain product line. 

In [19]:
data.groupby(["brand", "product_line"]).agg({"list_price":["mean", "count"]})
# OR using this code with pivot table we will get the same result:
data.pivot_table(index=["brand", "product_line"], values = "list_price", aggfunc =[np.mean, len])

mean        len
                              list_price list_price
brand          product_line                        
Giant Bicycles Road           951.094128      579.0
               Standard      1255.107490     2554.0
               Touring       1873.970000      179.0
Norco Bicycles Mountain       688.630000      200.0
               Road           817.646338      923.0
               Standard       987.691975     1787.0
OHM Cycles     Road           966.959089      779.0
               Standard       980.062010     2035.0
               Touring       1073.070000      229.0
Solex          Road          1323.944544      537.0
               Standard       955.575795     3508.0
               Touring       2083.940000      208.0
Trek Bicycles  Mountain       574.640000      223.0
               Road          1093.277648      995.0
               Standard      1311.262065     1772.0
WeareA2B       Road          1172.780000      157.0
               Standard      1188.195004     2520.0
               Touring       1593.374984      618.0

#### But if we change our code in pivot table adding columns data look much cleaner:


In [21]:
data.pivot_table(index=["brand"], columns= "product_line", values = "list_price", aggfunc =[np.mean, len], fill_value=0)

mean                                             len       \
product_line   Mountain         Road     Standard      Touring Mountain Road   
brand                                                                          
Giant Bicycles     0.00   951.094128  1255.107490  1873.970000        0  579   
Norco Bicycles   688.63   817.646338   987.691975     0.000000      200  923   
OHM Cycles         0.00   966.959089   980.062010  1073.070000        0  779   
Solex              0.00  1323.944544   955.575795  2083.940000        0  537   
Trek Bicycles    574.64  1093.277648  1311.262065     0.000000      223  995   
WeareA2B           0.00  1172.780000  1188.195004  1593.374984        0  157   

                                 
product_line   Standard Touring  
brand                            
Giant Bicycles     2554     179  
Norco Bicycles     1787       0  
OHM Cycles         2035     229  
Solex              3508     208  
Trek Bicycles      1772       0  
WeareA2B           2520     618

### Another example 

In [32]:
data.groupby(["brand", "product_line"]).agg({ "list_price":  ["mean"], "standard_cost": ["mean", "count"], })

# the same result with pivot without using columns

# data.pivot_table(index =["brand", 'product_line'], values = [ "standard_cost", 'list_price'], aggfunc={"list_price":[np.mean],"standard_cost":[np.mean, len]})


list_price standard_cost      
                                    mean          mean count
brand          product_line                                 
Giant Bicycles Road           951.094128    642.628394   579
               Standard      1255.107490    779.600388  2554
               Touring       1873.970000    863.950000   179
Norco Bicycles Mountain       688.630000    612.880000   200
               Road           817.646338    479.395103   923
               Standard       987.691975    685.326066  1787
OHM Cycles     Road           966.959089    784.537587   779
               Standard       980.062010    336.794776  2035
               Touring       1073.070000    933.840000   229
Solex          Road          1323.944544    900.446639   537
               Standard       955.575795    415.843769  3508
               Touring       2083.940000    675.030000   208
Trek Bicycles  Mountain       574.640000    459.710000   223
               Road          1093.277648    570.855377   995
               Standard      1311.262065    581.840705  1772
WeareA2B       Road          1172.780000   1043.770000   157
               Standard      1188.195004    438.507202  2520
               Touring       1593.374984    226.987702   618

In [37]:
# different view, we loosing side by side comparison
data.pivot_table(index =["brand"], columns='product_line', values = ['list_price', "standard_cost"], aggfunc={ "list_price":[len,np.median],"standard_cost":[np.median]}, fill_value =0)


list_price                                                   \
                      len                         median                     
product_line     Mountain Road Standard Touring Mountain     Road Standard   
brand                                                                        
Giant Bicycles          0  579     2554     179     0.00   792.90  1311.44   
Norco Bicycles        200  923     1787       0   688.63   774.53  1148.64   
OHM Cycles              0  779     2035     229     0.00  1280.28   912.52   
Solex                   0  537     3508     208     0.00  1703.52  1061.56   
Trek Bicycles         223  995     1772       0   574.64   980.37  1386.84   
WeareA2B                0  157     2520     618     0.00  1172.78  1231.15   

                        standard_cost                            
                               median                            
product_line    Touring      Mountain     Road Standard Touring  
brand                                                            
Giant Bicycles  1873.97          0.00   594.68   709.48  863.95  
Norco Bicycles     0.00        612.88   407.54   689.18    0.00  
OHM Cycles      1073.07          0.00   829.51   137.90  933.84  
Solex           2083.94          0.00   820.78   431.45  675.03  
Trek Bicycles      0.00        459.71   400.13   388.92    0.00  
WeareA2B        1466.68          0.00  1043.77   230.09  260.14

In [14]:
data.groupby(["brand", "product_line"]).agg({"product_line":["count"]})
# not very clear summary

product_line
                                   count
brand          product_line             
Giant Bicycles Road                  579
               Standard             2554
               Touring               179
Norco Bicycles Mountain              200
               Road                  923
               Standard             1787
OHM Cycles     Road                  779
               Standard             2035
               Touring               229
Solex          Road                  537
               Standard             3508
               Touring               208
Trek Bicycles  Mountain              223
               Road                  995
               Standard             1772
WeareA2B       Road                  157
               Standard             2520
               Touring               618

In [24]:
# while this one is much bettter!!
data.pivot_table(index="brand", columns="product_line", values = "list_price", aggfunc = len, fill_value = 0)

product_line,Mountain,Road,Standard,Touring
brand,,,,
Giant Bicycles,0,579,2554,179
Norco Bicycles,200,923,1787,0
OHM Cycles,0,779,2035,229
Solex,0,537,3508,208
Trek Bicycles,223,995,1772,0
WeareA2B,0,157,2520,618


#### More aggregation with pivot_table

In [43]:
cycle_summary= data.pivot_table(index=["brand", "product_size"], columns="product_line", values = ["list_price", "standard_cost"], aggfunc= {"list_price":[np.mean, sum]}, fill_value = 0)
cycle_summary

list_price                                         \
                                  mean                                          
product_line                  Mountain         Road     Standard      Touring   
brand          product_size                                                     
Giant Bicycles large              0.00     0.000000  1469.252182  1873.970000   
               medium             0.00  1135.750757  1027.704889     0.000000   
               small              0.00   590.260000  1643.548440     0.000000   
Norco Bicycles large              0.00  1023.324683     0.000000     0.000000   
               medium             0.00   653.264035   913.978455     0.000000   
               small            688.63     0.000000  1147.535301     0.000000   
OHM Cycles     large              0.00    12.010000     0.000000     0.000000   
               medium             0.00  1012.810365   980.062010  1073.070000   
               small              0.00  1810.000000     0.000000     0.000000   
Solex          large              0.00  1777.800000  1124.237386  2083.940000   
               medium             0.00   416.980000   913.530527     0.000000   
               small              0.00  1703.520000     0.000000     0.000000   
Trek Bicycles  large              0.00  1894.190000  1899.222987     0.000000   
               medium           574.64   610.598060   620.318294     0.000000   
               small              0.00  1720.700000  1582.354021     0.000000   
WeareA2B       large              0.00     0.000000     0.000000  1648.767209   
               medium             0.00     0.000000  1255.509656  1466.680000   
               small              0.00  1172.780000   828.222695     0.000000   

                                                                          
                                   sum                                    
product_line                  Mountain       Road    Standard    Touring  
brand          product_size                                               
Giant Bicycles large              0.00       0.00  1131324.18  335440.63  
               medium             0.00  434992.54  1431592.91       0.00  
               small              0.00  115690.96   642627.44       0.00  
Norco Bicycles large              0.00  419563.12        0.00       0.00  
               medium             0.00  335124.45  1117795.65       0.00  
               small         137726.00       0.00   647209.91       0.00  
OHM Cycles     large              0.00    2341.95        0.00       0.00  
               medium             0.00  388919.18  1994426.19  245733.03  
               small              0.00  362000.00        0.00       0.00  
Solex          large              0.00  325337.40   786966.17  433459.52  
               medium             0.00   70469.62  2565193.72       0.00  
               small              0.00  315151.20        0.00       0.00  
Trek Bicycles  large              0.00  431875.32  1278177.07       0.00  
               medium        128144.72  365137.64   447249.49       0.00  
               small              0.00  290798.30   598129.82       0.00  
WeareA2B       large              0.00       0.00        0.00  708969.90  
               medium             0.00       0.00  2665447.00  275735.84  
               small              0.00  184126.46   328804.41       0.00

In [46]:
cycle_summary.query('product_size == ["medium"]')

list_price                                     \
                                  mean                                      
product_line                  Mountain         Road     Standard  Touring   
brand          product_size                                                 
Giant Bicycles medium             0.00  1135.750757  1027.704889     0.00   
Norco Bicycles medium             0.00   653.264035   913.978455     0.00   
OHM Cycles     medium             0.00  1012.810365   980.062010  1073.07   
Solex          medium             0.00   416.980000   913.530527     0.00   
Trek Bicycles  medium           574.64   610.598060   620.318294     0.00   
WeareA2B       medium             0.00     0.000000  1255.509656  1466.68   

                                                                          
                                   sum                                    
product_line                  Mountain       Road    Standard    Touring  
brand          product_size                                               
Giant Bicycles medium             0.00  434992.54  1431592.91       0.00  
Norco Bicycles medium             0.00  335124.45  1117795.65       0.00  
OHM Cycles     medium             0.00  388919.18  1994426.19  245733.03  
Solex          medium             0.00   70469.62  2565193.72       0.00  
Trek Bicycles  medium        128144.72  365137.64   447249.49       0.00  
WeareA2B       medium             0.00       0.00  2665447.00  275735.84

In [49]:
cycle_summary.query("brand ==['Giant Bicycles']")

list_price                                     \
                                  mean                                      
product_line                  Mountain         Road     Standard  Touring   
brand          product_size                                                 
Giant Bicycles large               0.0     0.000000  1469.252182  1873.97   
               medium              0.0  1135.750757  1027.704889     0.00   
               small               0.0   590.260000  1643.548440     0.00   

                                                                        
                                 sum                                    
product_line                Mountain       Road    Standard    Touring  
brand          product_size                                             
Giant Bicycles large             0.0       0.00  1131324.18  335440.63  
               medium            0.0  434992.54  1431592.91       0.00  
               small             0.0  115690.96   642627.44       0.00